In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Reshape, LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import pairwise_distances
from scipy import linalg

# Load and preprocess your dataset (images) for training

# Define the Generator model
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(784, activation='sigmoid'))
    model.add(Reshape((28, 28)))
    return model

# Define the Discriminator model
def build_discriminator(img_shape):
    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Build and compile the GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential([generator, discriminator])
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
    return model

# Training loop
def train_gan(generator, discriminator, gan, dataset, latent_dim, epochs, batch_size):
    for epoch in range(epochs):
        for _ in range(dataset.shape[0] // batch_size):
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            generated_images = generator.predict(noise)
            real_images = dataset[np.random.randint(0, dataset.shape[0], batch_size)]

            labels_real = np.ones((batch_size, 1))
            labels_fake = np.zeros((batch_size, 1))

            d_loss_real = discriminator.train_on_batch(real_images, labels_real)
            d_loss_fake = discriminator.train_on_batch(generated_images, labels_fake)

            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            labels_gan = np.ones((batch_size, 1))
            g_loss = gan.train_on_batch(noise, labels_gan)

        print(f'Epoch {epoch}/{epochs} - D Loss Real: {d_loss_real} - D Loss Fake: {d_loss_fake} - G Loss: {g_loss}')

        if (epoch + 1) % save_interval == 0:
            save_generated_images(generator, epoch)

# MiFID Score Calculation
def calculate_mifid_score(real_images, generated_images, num_samples=10000):
    def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
        covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
        if not np.isfinite(covmean).all():
            msg = f'fid calculation produces singular product; adding {eps} to diagonal of cov estimates'
            print(msg)
            offset = np.eye(sigma1.shape[0]) * eps
            covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))
        # Numerical error might give slight imaginary component
        if np.iscomplexobj(covmean):
            if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
                m = np.max(np.abs(covmean.imag))
                raise ValueError(f'Imaginary component {m}')
            covmean = covmean.real
        tr_covmean = np.trace(covmean)
        return np.dot(mu1 - mu2, mu1 - mu2) + np.trace(sigma1) + np.trace(sigma2) - 2 * tr_covmean

    num_batches = min(len(real_images), len(generated_images)) // num_samples
    real_images = real_images[:num_samples * num_batches]
    generated_images = generated_images[:num_samples * num_batches]

    real_images = real_images.reshape(num_batches, num_samples, -1)
    generated_images = generated_images.reshape(num_batches, num_samples, -1)

    real_mu = np.mean(real_images, axis=1)
    real_sigma = np.cov(real_images, rowvar=False)

    fake_mu = np.mean(generated_images, axis=1)
    fake_sigma = np.cov(generated_images, rowvar=False)

    mifid_score = calculate_frechet_distance(real_mu, real_sigma, fake_mu, fake_sigma)
    return mifid_score

# Main training and evaluation loop
if __name__ == '__main__':
    latent_dim = 100
    img_shape = (28, 28)
    epochs = 10000
    batch_size = 128
    save_interval = 1000

    # Load and preprocess your dataset (images) for training

    # Build and compile models
    generator = build_generator(latent_dim)
    discriminator = build_discriminator(img_shape)
    gan = build_gan(generator, discriminator)

    # Load and preprocess your dataset (images) for training

    # Train the GAN
    train_gan(generator, discriminator, gan, dataset, latent_dim, epochs, batch_size)

    # Generate a batch of fake images for MiFID calculation
    num_samples_for_mifid = 10000
    noise_for_mifid = np.random.normal(0, 1, (num_samples_for_mifid, latent_dim))
    generated_images_for_mifid = generator.predict(noise_for_mifid)

    # Calculate MiFID Score
    mifid_score = calculate_mifid_score(real_images, generated_images_for_mifid)
    print(f'MiFID Score: {mifid_score}')
